In [ ]:
import pickle as pkl
import spacy
import re
import torchtext.vocab
from torchtext.vocab import build_vocab_from_iterator
from os.path import exists
import os
import torch
import sys

In [ ]:
def tokenize(text, tokenizer):
    x = 0
    return [tok.text for tok in tokenizer.tokenizer(text)]

In [ ]:
def yield_tokens(data_iter, tokenizer):
    for line in data_iter:
        yield tokenizer(line)

In [ ]:
def build_vocabulary_from_file(spacy_en, filename: str, lowercase=True):
    def tokenize_en(text):
        return tokenize(text, spacy_en)

    print(f"Building English Vocabulary from {filename} ...")
    # train, val, test = datasets.Multi30k(language_pair=("de", "en"))
    with open(filename, encoding="utf-8") as f:
        if lowercase:
            train = f.read().lower().splitlines()
        else:
            train = f.read().splitlines()
    vocab = build_vocab_from_iterator(
        yield_tokens(train, tokenize_en),
        min_freq=1,
        specials=["<s>", "</s>", "<blank>", "<unk>"],
    )

    vocab.set_default_index(vocab["<unk>"])

    return vocab

In [ ]:
def load_vocab(filename: str):
    if not exists(filename):
        raise Exception
    else:
        vocab = torch.load(filename)
    print("Finished.\nVocabulary sizes:")
    print(len(vocab))
    return vocab

In [ ]:
def build_and_save_vocab_from_file(nlp, txt_filename, vocab_save_filename):
    vocab = build_vocabulary_from_file(nlp, txt_filename)
    torch.save((vocab), vocab_save_filename)
    return vocab

In [ ]:
def get_vocab_indx_vector(vocab, tokenizer, text):
    return vocab([tok.text for tok in tokenizer.tokenizer(text)])

In [ ]:
def decode_vocab(vocab: torchtext.vocab.Vocab, tokens_ind):
    return vocab.lookup_tokens(tokens_ind)

In [ ]:
def generate_tokenized_file(vocab, tokenizer,  txt_filename: str, tok_filename: str, lowercase=False):
    with open(txt_filename, mode="r", encoding="utf-8") as txt_f:
        if lowercase:
            complete_txt = " ".join(filter(None, txt_f.read().lower().splitlines()))
        else:
            complete_txt = " ".join(filter(None, txt_f.read().splitlines()))
        with open(tok_filename, mode="wb") as tok_f:
            tokenized_txt = get_vocab_indx_vector(vocab, tokenizer, complete_txt)
            pkl.dump(tokenized_txt, tok_f)

In [ ]:
vocab = build_and_save_vocab_from_file(tokenizer, "", "")
generate_tokenized_file(vocab, tokenizer, filename, output_file, lowercase=True)
